Análise do sistema 12C+24Mg para a disciplina de Reações Nucleares

http://www.dfn.if.usp.br/~rubens/reacoes/

In [45]:
# ======== IMPORTS ======== #

import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [68]:
# ======== LEITURA DA BASE DE DADOS ======== #

def base(): # Cria lista com os nomes dos arquivos de entrada
            # Calcula itens 2, 3

    os.system('cd Dados\nls > ls.txt')
    ls = open('Dados/ls.txt', 'r')

    cmg = []
    E = []


    for line in ls:

        line = line.replace('\n', '')
        if line == 'ls.txt': break

        cmg += [line]
        E += [float(line.replace('cmg', ''))/10]

    return cmg, E

def leia(nome): # Le os dados do sistema e faz uma tabela

    arc = open('Dados/' + nome, 'r')

    Ang = []
    Sig = []
    Inc = []

    for line in arc:
        
        line = line.replace('\n', '')
        line = line.split(' ')
        line = list(filter(None, line))

        if len(line) < 3: break

        Ang += [float(line[0])]
        Sig += [float(line[1])]
        Inc += [float(line[2])]

    df = pd.DataFrame({'Theta':Ang, 'Sigma':Sig, 'Incerteza':Inc})

    return df

def grafico(df, E):

    x = np.array(df['Theta'])
    y = np.array(df['Sigma'])
    inc = np.array(df['Incerteza'])

    plt.scatter(x, y, zorder = 1, s = 20, color = 'black')

    # Incerteza
    barra = 1

    for i in range(len(df)):
        plt.plot([x[i], x[i]], [y[i]-inc[i], y[i]+inc[i]], color = 'black', zorder = 0)
        plt.plot([x[i]-barra, x[i]+barra], [y[i]+inc[i], y[i]+inc[i]], color = 'black', zorder = 0)
        plt.plot([x[i]-barra, x[i]+barra], [y[i]-inc[i], y[i]-inc[i]], color = 'black', zorder = 0)

    plt.title('$^{12}$C+$^{24}$Mg @ E$_{lab}$=' + str(E) + 'MeV')

    # plt.yscale('log')
    # plt.ylim(0.01, 10)
    plt.ylim(0, 10)

    plt.xlim(0, 180)
    plt.xlabel('Theta$_{CM}$ [deg]')
    plt.grid()
    plt.ylabel('Sigma [mb/sr]')
    plt.savefig('Graficos/' + str(int(E*10)))



In [86]:
# ======== CALCULOS ======== #

def VbCalc(Z1, Z2, A1, A2): # Barreira Coulombiana

    e2 = 1.44
    Rb = 1.07*(A1**(1/3)+A2**(1/3)) + 2.72

    Vb = Z1*Z2*e2/Rb
    return Vb

def LgCalc(A1, A2, Vb, E): # Momento Angular Grazing

    mu = A1*A2/(A1+A2)
    R = 1.2*(A1**(1/3)+A2**(1/3))
    kl = np.sqrt(2*mu*(E-Vb))

    Lg = kl*R
    return Lg

def TgCalc(Z1, Z2, A1, E, Lg):

    e2 = 1.44
    h = 1
    v = np.sqrt(2*E/A1)
    eta = Z1*Z2*e2/(h*v)

    Tg = 2*np.arctan(eta/Lg)
    return Tg

def SigmaCalc(A1, A2, E, Vb, Lg):

    mu = A1*A2/(A1+A2)
    kl = np.sqrt(2*mu*(E-Vb))
    Rb = 1.07*(A1**(1/3)+A2**(1/3)) + 2.72
    Sigma = np.pi*Lg**2/kl**2

    Sigma = np.pi*Rb**2*(1-Vb/E)
    return Sigma

In [87]:
# ======== FUNCAO MAIN ======== #

def main():

    Dec = 4 # Numero de casas decimais

    # 12C + 24Mg
    Z1 = 6
    A1 = 12
    Z2 = 12
    A2 = 6

    # ========================================= #
    # 1) Calculo de Vb

    Vb = round(VbCalc(Z1, Z2, A1, A2), Dec)
    print('1) Vb = %s MeV' %Vb)
    print()
    print('R', str(1.2*(A1**(1/3)+A2**(1/3))))
    print('Rb', str(1.07*(A1**(1/3)+A2**(1/3)) + 2.72))
    
    # ========================================= #
    # 2) Calculo de Lg

    cmg, E = base() # Lista de energias

    Lg = [LgCalc(A1, A2, Vb, Ei) for Ei in E]

    # ========================================= #
    # 3) Calculo da funcao de deflexao Coulombiana

    Tg = [TgCalc(Z1, Z2, A1, E[i], Lg[i]) for i in range(len(E))]

    # ========================================= #
    # 5) Calculo do Sigma de reacao

    Sig = [SigmaCalc(A1, A2, E[i], Vb, Lg[i]) for i in range(len(E))]

    # ========================================= #
    # Resultados

    result = pd.DataFrame({'Energia':E, 'Lg':Lg, 'Tg':Tg, 'Sigma':Sig})

    display(result)

    # for nome in cmg:

    #     df = leia(nome)
    #     break

main()

1) Vb = 14.5741 MeV

R 4.9278588935265635
Rb 7.114007513394521


,Energia,Lg,Tg,Sigma
0,16.0,16.643620,2.628845,14.169274
1,17.0,21.708994,2.463887,22.688328
2,18.0,25.798271,2.327744,30.260820
3,18.5,27.616769,2.266605,33.740073
4,19.0,29.322707,2.209055,37.036207
5,19.5,30.934710,2.154598,40.163309
6,20.0,32.466775,2.102861,43.134056
7,20.5,33.929731,2.053557,45.959889
8,21.0,35.332164,2.006457,48.651158
9,21.5,36.681017,1.961373,51.217251


In [82]:
print(np.pi)

3.141592653589793
